# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [56]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [57]:
# 1) Seu código aqui

url = ' http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)

In [58]:
r.status_code

200

In [59]:
data_json = r.json()

In [60]:
df = pd.DataFrame(data_json['items'])

In [61]:
df.dtypes

id_pleito                  int64
tipo_interessado          object
interessado               object
cod_ibge                   int64
uf                        object
num_pvl                   object
status                    object
num_processo              object
data_protocolo            object
tipo_operacao             object
finalidade                object
tipo_credor               object
credor                    object
moeda                     object
valor                    float64
pvl_assoc_divida           int64
pvl_contradado_credor      int64
data_status               object
dtype: object

In [62]:
# 2) Seu código aqui

df['status'].value_counts()

status
Deferido                                                 106
Arquivado                                                 53
Arquivado por decurso de prazo                            26
Regularizado                                              26
Deferido (PVL-IF)                                         26
Encaminhado à PGFN com manifestação técnica favorável     20
Arquivado a pedido                                         8
Em retificação pelo interessado                            6
Assinado pelo interessado (retificação)                    4
Em análise                                                 2
Em retificação pelo credor                                 2
Em retificação pelo interessado (PVL-IF)                   1
Arquivado pela STN                                         1
Arquivado a pedido (PVL-IF)                                1
Name: count, dtype: int64

In [63]:
# 3) Seu código aqui

# Primeiro, converto a coluna data_status para string
df['data_status'] = df['data_status'].astype(str)

# Agora, crio uma nova coluna ('ano_status') extraindo o ano
df['ano_status'] = df['data_status'].str.split('/').str[-1]

# Pra terminar, converto a coluna 'ano_status' para int
df['ano_status'] = df['ano_status'].astype(int)

In [64]:
# Abaixo indico a frequência de cada ano do campo construído no item (3).

df['ano_status'].value_counts()

ano_status
2008    37
2007    29
2023    28
2014    20
2010    19
2025    17
2019    16
2013    15
2020    14
2012    13
2022    11
2011    10
2009     9
2015     8
2024     8
2021     5
2017     5
2016     4
2018     4
2006     4
2002     4
2003     1
2004     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [65]:
#1) Seu código aqui

def obter_dados(uf: str, tipo_interessado: str) -> pd.DataFrame:
    uf = uf.strip().upper()
    tipo = tipo_interessado.strip().capitalize()

    if tipo not in ['Estado', 'Município']:
        raise ValueError("tipo_interessado deve ser 'Estado' ou 'Município'")

    url = "http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    params = {
        "uf": uf,
        "tipo_interessado": tipo
    }

    resp = requests.get(url, params=params, timeout=10)
    resp.raise_for_status()

    dados_json = resp.json()

    if 'items' in dados_json:
        lista = dados_json['items']
    else:
        lista = dados_json

    df = pd.DataFrame(lista)
    return df

In [73]:
# 2) Seu código aqui

df_mg = obter_dados('MG', 'Estado')
df_mg.head(5)

df_mg['status'].value_counts()['Arquivado por decurso de prazo']

np.int64(1)

In [84]:
# 3) Seu código aqui

df_ba_mun = obter_dados('BA', 'Município')
df_ba_mun.head(5)

mais_frequente = df_ba_mun['interessado'].value_counts().idxmax()
frequencia = df_ba_mun['interessado'].value_counts().max()

print(f"O município com mais solicitações deferidas é {mais_frequente} e aparece {frequencia} vezes.")

O município com mais solicitações deferidas é Camaçari e aparece 21 vezes.


In [91]:
# 4) Seu código aqui

df_ba = obter_dados('BA', 'Estado')
df_ba.head(5)

df_ba.to_csv('df_ba.csv', index = False)